In [1]:
%cd ~/Dropbox/Project1_N4526/sextractor/DM_analysis

/Users/lchevalier/Dropbox/Project1_N4526/sextractor/DM_analysis


In [4]:
import numpy as np
import pandas as pd
import itertools
import scipy.stats
import sys,os,commands
from scipy.special import gammainc as GammaINC
from os import path
from numpy import append
from scipy.special import gamma as Gamma
import math
from scipy import optimize
from scipy.optimize import fmin_powell
from numpy import random


def dist(x1,y1,x2,y2):
    return np.hypot((x2-x1),(y2-y1))


def outliers_show(RV, RAD, RA, DEC):
    #clean data set!, remove outliers using friendless algorithm by Merrett et al 2003.
    N_nn= int(np.sqrt(len(RV)))
    tmp = []
    for i in range(len(RV)):
        temp = []
        for j in range(len(RV)):
            if j != i:
                temp.append(dist(RA[i], DEC[i], RA[j], DEC[j]))
        
        sep=np.asarray(temp)
        idx=np.argsort(sep)
        n_idx=idx[:N_nn]
        loc_vel_ave = np.average(RV[n_idx])
        loc_vel_dis = np.std(RV[n_idx])
        if np.abs(RV[i]-loc_vel_ave) > 3.0 * loc_vel_dis:
            tmp.append(i)
    f_idx = np.asarray(tmp)
    if len(f_idx) > 0 :
        print "there are outliers in the dataset, your attention is needed!"
    return f_idx

def afunc(x,paos,v_sys,q_phot):
    pi=math.pi
    #global ros,paos,rvos,veos
    #global k,pi
    #global v_sys,q_phot,pa_phot,rad
    vrotc=x[0]
    sigmac=x[1]
    pakinc=x[2]
    # Sanity check that parameters are within reasonable range.
    if sigmac <0. or sigmac > 300.:
        return 999999.
    if pakinc <0. or pakinc>2.*pi:
        return 999999.
    if vrotc < 0. or vrotc > 300.:
        return 999999.
    lh=0.
    paos=paos/180.*pi
    for i in range(0,ngc):
        theta=paos[i]-pakinc
        while ((theta < 0.) or (theta > (2.*pi))) :
            if (theta<0.):
                theta=theta+2*pi
            elif (theta>(2.*pi)):
                theta=theta-2*pi
        if theta >= 0. and theta < (pi/2.) :
            v_exp=v_sys+vrotc/(math.sqrt(1.+(math.tan(theta)/q_phot)**2))
        elif theta >= (3.*pi/2.) and theta <= (2.*pi):
            v_exp=v_sys+vrotc/(math.sqrt(1.+(math.tan(theta)/q_phot)**2))
        else:
            v_exp=v_sys-vrotc/(math.sqrt(1.+(math.tan(theta)/q_phot)**2))
        lh=lh+((rvos[i]-v_exp)**2)/(sigmac**2+rveos[i]**2)+math.log(sigmac**2+rveos[i]**2)
    paos=paos/pi*180.
    return lh

def rotation_definition(file='ALL_rad_roll',percentile=50,columns =[2,3]):
    Vrot, sigma = np.loadtxt(file, unpack=True, usecols=columns)
    rot = Vrot/sigma
    #print scipy.stats.scoreatpercentile(Vrot[:], 50), scipy.stats.scoreatpercentile(Vrot[:], 50)-scipy.stats.scoreatpercentile(Vrot[:], 16), scipy.stats.scoreatpercentile(Vrot[:], 84)-scipy.stats.scoreatpercentile(Vrot[:], 50)
    #print scipy.stats.scoreatpercentile(sigma[:], 50), scipy.stats.scoreatpercentile(sigma[:], 50)-scipy.stats.scoreatpercentile(sigma[:], 16), scipy.stats.scoreatpercentile(sigma[:], 84)-scipy.stats.scoreatpercentile(sigma[:], 50)
    #print scipy.stats.scoreatpercentile(rot[:], 50), scipy.stats.scoreatpercentile(rot[:], 50)-scipy.stats.scoreatpercentile(rot[:], 16), scipy.stats.scoreatpercentile(rot[:], 84)-scipy.stats.scoreatpercentile(rot[:], 50)

    return scipy.stats.scoreatpercentile(Vrot[:], percentile),scipy.stats.scoreatpercentile(sigma[:], percentile),scipy.stats.scoreatpercentile(rot[:], percentile)


def Watkins(RAD, VELSQ, alpha, gamm, beta, item):
    rout = item #from Limber & Mathews+1960, An+2011(np.pi/4)
    for param1 in alpha:
        RAD = np.power(RAD, param1)
        mass_fac = np.average(np.multiply(RAD, VELSQ))
        for param2 in gamm:
            for param3 in beta:
                num_coeff = (param1+param2-2.*param3)*4.*Gamma(0.5*param1+2.5)*rout**(1.-param1)
                den_coeff = np.sqrt(np.pi)*(param1+3.-param3*(param1+2))*Gamma(0.5*param1+1.)
                coeffTME = num_coeff/den_coeff/4.3e-6
    return coeffTME * mass_fac

def define_masses_at_re( input_array,mode,re_sampled=np.arange(1.,45.,1.)):
    ''' the modes availible are iso, rad and tan. Not other modes availible.
    They stand for isotropy, radial and tangential orbits. Results should not differ from Busolas 2017 results.
    '''
    masses_at_re =[]
    err_masses_at_re =[]
    rmax = input_array[-1]*corr
    for i in re_sampled:
    #extract mass @  5 R_e, 8 R_e and at max rad
        idx0 = np.nonzero(rad > i*r_e)[0][0]

    #print mass estimates; iso, rad and tan
        #print("%4s \t %.2e \t %.2e \t %.2f \t %.2e \t %.2e" % ('4526', mass_W_I[idx0]*corr, mass_err_W_I[idx0], rad[-1]/r_e, mass_W_I[-1]*corr, mass_err_W_I[-1]))
        masses_at_re.append(input_array[idx0]*corr)
        err_masses_at_re.append(input_array[idx0])
        
        #print("%4s \t %.2e \t %.2e \t %.2f \t %.2e \t %.2e" % ('4526', mass_W_R[idx0]*corr, mass_err_W_R[idx0], rad[-1]/r_e, mass_W_R[-1]*corr, mass_err_W_R[-1]))
        #print("%4s \t %.2e \t %.2e \t %.2f \t %.2e \t %.2e" % ('4526', mass_W_T[idx0]*corr, mass_err_W_T[idx0], rad[-1]/r_e, mass_W_T[-1]*corr, mass_err_W_T[-1]))
        #print('##################################################################################')
    return masses_at_re, err_masses_at_re, rmax

def get_mass(r_e, max_rad, Vrot, err_l, err_u, mdyn, mdyn_err, corr):

    N=10000
    m_rot5 = max_rad * r_e * Vrot**2/4.302e-6
    mass_1 = np.empty(N)
    mass_3 = np.empty(N)
    for i in range(N):
        first = np.random.normal(Vrot, 0.5*err_l, 1)
        second = np.random.normal(mdyn, mdyn_err, 1)
        third = np.random.normal(max_rad, max_rad*0.15, 1)
        mass_1[i] = max_rad  * first**2/4.302e-6
        mass_3[i] = mass_1[i] + second*corr
    l_err_rot = np.percentile(mass_1[i], 50)-np.percentile(mass_1[i], 16)
    u_err_rot = np.percentile(mass_1[i], 84)-np.percentile(mass_1[i], 50)
    err_rot = np.mean([l_err_rot, u_err_rot])
    return (m_rot5, np.std(mass_1),  m_rot5+mdyn, np.std(mass_3))


def get_f_dm(M_star, err, max_rad, tot_mass5, tot_mass5_err, r_e, n_S):
    N=1000
    b = 2*n_S-(1./3.)+(0.009876/n_S)+(0.0018/n_S**2)
    p = 1.-(0.6097/n_S)+(0.05563/n_S**2)
    M_star5 = M_star * GammaINC((3.-p)*n_S, b*(1.34*max_rad)**(1./n_S))
    real_fdm5 =  1.-(M_star5/tot_mass5)
    real_MDM_Mstar5 =  tot_mass5/M_star5
    real_rhodm5 =  3.*(tot_mass5-M_star5)/(4.*np.pi*(max_rad*r_e)**3)

    f_dm5 = np.empty(N)
    MDM_Mstar5 = np.empty(N)
    rhodm5 = np.empty(N)

    for i in range(N):
        second = np.random.normal(tot_mass5, tot_mass5_err, 1)
        fifth = np.random.normal(M_star5, err, 1)
        f_dm5[i] = 1-(fifth/second)
        MDM_Mstar5[i] = second/fifth
        rhodm5[i] = 3.*(second-fifth)/(4.*np.pi*(max_rad*r_e)**3)
    return (real_fdm5, np.std(f_dm5))

In [6]:
def fDM():
    input_file='NGC4526.cat'
    friendless=True
    mode='iso'
    #def calculate_dark_matter_fraction(RV_file):
    #input_file=RV_file
    all_rad_file='ALL_rad_roll_'+input_file
    mass_estW_ISO='mass_estW_ISO_'+input_file
    mass_estW_RAD='mass_estW_RAD_'+input_file
    mass_estW_TAN='mass_estW_TAN_'+input_file


    out=open(all_rad_file,'w')
    out.write('#it \t\t rad \t\t Vrot \t\t sigma \t\t PAkin \t\t lh/dof \n')

    nbs=30
    v_sys=617.
    pi=math.pi
    q_phot=0.24
    pa_phot=(113.7)*pi/180.0
    RA0 = 188.512856
    DEC0 = 7.699524
    Vsys = 617.
    Q = 0.24
    PA0 = 113.7
    Dist = 16.4
    Re_arc = 32.4
    Re_kpc = 2.576
    M_star = 11.26

    corr=0.77
    gal = 'N4526'
    r_e = 2.576 #Re in kpc, Duncan's paper
    n_S = 3.6 #Sersic Index, Duncan's paper

    data = np.loadtxt(input_file, dtype={'names':('ID', 'RA', 'DEC', 'RV', 'RV_err', 'col', 'PA', 'RAD'), 'formats':('|S20', np.float, np.float, np.float, np.float, np.float, np.float, np.float)})

    ID = data['ID']
    RA =data['RA']
    DEC=data['DEC']
    RV=data['RV']
    RV_err=data['RV_err']
    col=data['col']
    RAD=data['RAD']
    PA=data['PA']


    if friendless==True:
        print "Welcome, rot-sig estimation routine!"
        print "+++++++++++++++++++++++++++++++++"
        print ""
        print "There are ", len(ID), "objects in your dataset, before cleaning"

        no_print = outliers_show(RV, RAD, RA, DEC)
        fRAD = np.empty(len(no_print))
        fRV = np.empty(len(no_print))

        for i in range(len(no_print)):
            print ID[no_print[i]]
            fRAD[i] = RAD[no_print[i]]
            fRV[i] = RV[no_print[i]]

        #clean friendless objects from parent dataset, and update (not writing to file!!!)
        ID = np.delete(ID, list(no_print))
        RA = np.delete(RA, list(no_print))
        DEC = np.delete(DEC, list(no_print))
        RV = np.delete(RV, list(no_print))
        RV_err = np.delete(RV_err, list(no_print))
        col = np.delete(col, list(no_print))
        PA = np.delete(PA, list(no_print))
        RAD = np.delete(RAD, list(no_print))


        print "dataset now has ", len(ID), "objects, after cleaning"
        #return(ID,RA,DEC,RV,RV_err,col,PA,RAD)
    ngc = len(RA)
    print 'checkpoint 0'
    # ORDERING BY GALACTOCENTRIC RADIUS:            #kin.py
    # All GCs:          #kin.py
    global ros,paos,rvos,rveos 
    inds=np.argsort(RAD)            #kin.py
    ros=RAD[inds]           #kin.py
    paos=PA[inds]           #kin.py
    rvos=RV[inds]           #kin.py
    rveos=RV_err[inds]          #kin.py
               #kin.py
    # INITIAL GUESS:            #kin.py
    x0=np.zeros(3)          #kin.py
    x0[0]=80.           #kin.py
    x0[1]=200.          #kin.py
    x0[2]=pa_phot           #kin.py

    # Bootstrapping:
    randind=[]
    for i in range(0,ngc):
        randind.append(0)

    for j in range (0,nbs-1):
        #print j
        # Randomizing all GCs:
        for i in range(0,ngc):
            randind[i]=random.randint(0,ngc) # assigning random index
        randrs=RAD[randind]
        randpas=PA[randind]
        randrvs=RV[randind]
        randrves=RV_err[randind]

        # ORDERING BY GALACTOCENTRIC RADIUS:
        # ALL GCs:
        inds=np.argsort(randrs)
        ros=randrs[inds]
        paos=randpas[inds]
        rvos=randrvs[inds]
        rveos=randrves[inds]

        # INITIAL GUESS:
        x0=np.zeros(3)
        x0[0]=60.
        x0[1]=130.
        x0[2]=pi

    # MINIMIZING USING POWELL'S METHOD for all GCs:
        rad=0.
        rad=sum(ros[0:ngc])/float(ngc)
        
        #paos,k,v_sys,q_phot
        #paos=paos,k=k,v_sys=v_sys,q_phot=q_phot
        
        powellout=fmin_powell(afunc,x0,args=(paos,v_sys,q_phot),ftol = 1e-6, xtol=1e-6, maxiter = None,full_output=1,disp=0)
        xopt=powellout[0]
        #using last fit as new initial guess:
        x0=xopt
        #print j+1,rad,xopt[0],xopt[1],(xopt[2]/pi*180.),powellout[1]/float(ngc-3)
        out.write(str(j+1)+'\t'+str(rad)+'\t'+str(xopt[0])+'\t'+str(xopt[1])+'\t'+str((xopt[2]/pi*180.))+'\t'+str(powellout[1]/float(ngc-3))+'\n')
    print 'checkpoint 1'
    #out1=open('FRIENDLESS','w')
    #out1.write('#ID \t RA \t DEC \t RV \t RV_err \t col \t PA \t RAD \t \n')
    out.close()
    out4=open(mass_estW_ISO,'w')
    out4.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')
    out5=open(mass_estW_RAD,'w')
    out5.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')
    out6=open(mass_estW_TAN,'w')
    out6.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')


    #det gamm and alpha parameter
    gamm_param = -0.63*M_star+9.81
    alp_param = -0.46*M_star+5.29
    #define alpha, beta and gamma
    gamm = [gamm_param]
    alpha = [alp_param]
    beta = [[0.], [0.5], [-0.5]]

    from tqdm import trange
    print 'checkpoint 2'
    Vrot = rotation_definition(file=all_rad_file,percentile=50,columns =[2,3])[0]#88.57
    err_l = rotation_definition(file=all_rad_file,percentile=16,columns =[2,3])[0]#27.8421455382 #16th percentile
    err_u = rotation_definition(file=all_rad_file,percentile=84,columns =[2,3])[0] #84th percentile

    RV -= Vsys
    RV -= Vrot
    RAD = RAD * Re_kpc/Re_arc
    #sort data by RAD before analysis
    indx = np.argsort(RAD)
    RV = RV[indx]
    RV_err = RV_err[indx]
    RAD = RAD[indx]

    N=1000 #number of iterations for Montecarlo simulations
    idx = np.nonzero(RAD > 0.5*Re_kpc)[0] #returns index of GCs greater than 2R_e
    print idx, RAD.min(),RAD.max(),Re_kpc
    mass_est_err0 = np.ones(N)
    mass_est_err1 = np.ones(N)
    mass_est_err2 = np.ones(N)
    print 'checkpoint 3'
    for rad_lim in RAD[idx]:
        aRAD = RAD[np.where(np.logical_and(RAD < rad_lim, 1))]
        aRV = RV[np.where(np.logical_and(RAD < rad_lim, 1))]
        aRV_err = RV_err[np.where(np.logical_and(RAD < rad_lim, 1))]
        #real_mass, without Monte Carlo simulations
        real_mass_est0 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[0], rad_lim)
        real_mass_est1 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[1], rad_lim)
        real_mass_est2 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[2], rad_lim)
        #Now Monte Carlo, that makes use of RV_err and sampling without replacement
        for j in range(N):
            scr_RV = np.ones(len(aRV))
            for i in range(len(aRV)):
                scr_RV[i] = np.random.normal(aRV[i], aRV_err[i], 1)
            VELSQ = np.square(scr_RV)
            mass_est_err0[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[0], rad_lim)/np.sqrt(len(aRAD))
            mass_est_err1[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[1], rad_lim)/np.sqrt(len(aRAD))
            mass_est_err2[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[2], rad_lim)/np.sqrt(len(aRAD))
        #print ("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est0, scipy.stats.scoreatpercentile(mass_est_err0[:], 16), scipy.stats.scoreatpercentile(mass_est_err0[:], 84)))
        out4.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est0, scipy.stats.scoreatpercentile(mass_est_err0[:], 16), scipy.stats.scoreatpercentile(mass_est_err0[:], 84)))
        out5.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est1, scipy.stats.scoreatpercentile(mass_est_err1[:], 16), scipy.stats.scoreatpercentile(mass_est_err1[:], 84)))
        out6.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est2, scipy.stats.scoreatpercentile(mass_est_err2[:], 16), scipy.stats.scoreatpercentile(mass_est_err2[:], 84)))
    out4.close()
    out5.close()
    out6.close()
    #######################################
    # find the masses and max re from the following code!! Also see output.py!!

    #######################################
    #read input files
    #isotropy

    ngc, rad, mass_W_I, lmass_err_W_I, umass_err_W_I = np.loadtxt(mass_estW_ISO, unpack=True, usecols=[0,1,2,3,4])
    mass_err_W_I = np.maximum(lmass_err_W_I, umass_err_W_I)
    #radial orbits
    mass_W_R, lmass_err_W_R, umass_err_W_R = np.loadtxt(mass_estW_RAD, unpack=True, usecols=[2,3,4])
    mass_err_W_R = np.maximum(lmass_err_W_R, umass_err_W_R)
    #tangential orb
    mass_W_T, lmass_err_W_T, umass_err_W_T = np.loadtxt(mass_estW_TAN, unpack=True, usecols=[2,3,4])
    mass_err_W_T = np.maximum(lmass_err_W_T, umass_err_W_T)


    effective_radii_to_investigate=np.append(np.arange(3.,(rad[-1]/r_e),1.),np.round(rad[-1]/r_e,2))
    max_rad = effective_radii_to_investigate
    # Isotropic 
    if mode == 'iso':
        m5 = define_masses_at_re(input_array=mass_W_I,re_sampled=effective_radii_to_investigate, mode='iso')[0]#2.66e+11#mass within 5/8/Rmax 3.52e+11 8.35e+10 
        m5err = define_masses_at_re(input_array=mass_err_W_I,re_sampled=effective_radii_to_investigate,mode='iso')[1]#7.86e+10 #uncertainty

    # rad 
    elif mode == 'rad':
        m5 = define_masses_at_re(effective_radii_to_investigate, mode='rad')[0]
        m5err = define_masses_at_re(effective_radii_to_investigate,mode='rad')[1]

    #tan
    elif mode == 'tan':
        m5 = define_masses_at_re(effective_radii_to_investigate, mode='tan')[0]
        m5err = define_masses_at_re(effective_radii_to_investigate,mode='tan')[1]



    M_star = 11.26
    M_star = 10**M_star
    mstar_err = 0.10*M_star
    dm_frac = []
    dm_frac_err = []
    mass_re=[]
    mass_re_max=[]
    mass_re_err=[]
    mass_re_max_err=[]
    #obtain parameters and uncertainties on calculated parameters
    ######################## Needs to loop over all the Vrot values
    for i in range(len(max_rad)):
        m_rot5, m_rot5_err, tot_mass5, tot_mass5_err = get_mass(r_e, max_rad[i], Vrot, err_l, err_u, m5[i], m5err[i], corr)
        f_dm5, f_dm5_err = get_f_dm(M_star, mstar_err, max_rad[i], tot_mass5, tot_mass5_err, r_e, n_S)
        dm_frac.append(f_dm5)
        dm_frac_err.append(f_dm5_err)
        mass_re.append(m_rot5)
        mass_re_max.append(tot_mass5)
        mass_re_err.append(m_rot5_err)
        mass_re_max_err.append(tot_mass5_err)
        print("%5s \t %1s \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2f \t %.2f \n" % (str(gal),max_rad[i],m_rot5, m_rot5_err, m5[i], m5err[i], tot_mass5, tot_mass5_err, f_dm5, f_dm5_err))
    #"%5s \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2f \t %.2f \n" %
    #4526    2.66e+11    7.86e+10    16.75   6.52e+11    1.35e+11
    #return (max_rad, m_rot5, m_rot5_err, m5[i], m5err[i], tot_mass5, tot_mass5_err, f_dm5, f_dm5_err)
    d ={'re': effective_radii_to_investigate,
        'fDM': dm_frac,
        'fDM_err': dm_frac_err,
        'mtot':tot_mass5,
        'mtot_err':tot_mass5_err,
        'mass_rot':mass_re,
        'mass_rot_err': mass_re_err,
        'mass_press': m5,
        'mass_press_err': m5err}
    df = pd.DataFrame(d)

    df.to_csv(str(input_file[:-4])+'_'+str(friendless)+'_'+str(mode)+'.csv')
    return df

In [8]:
input_file='N4526_lowRV_highDist.txt'
friendless=True
mode='iso'
#def calculate_dark_matter_fraction(RV_file):
#input_file=RV_file
all_rad_file='ALL_rad_roll_'+input_file
mass_estW_ISO='mass_estW_ISO_'+input_file
mass_estW_RAD='mass_estW_RAD_'+input_file
mass_estW_TAN='mass_estW_TAN_'+input_file


out=open(all_rad_file,'w')
out.write('#it \t\t rad \t\t Vrot \t\t sigma \t\t PAkin \t\t lh/dof \n')

nbs=30
v_sys=617.
pi=math.pi
q_phot=0.24
pa_phot=(113.7)*pi/180.0
RA0 = 188.512856
DEC0 = 7.699524
Vsys = 617.
Q = 0.24
PA0 = 113.7
Dist = 16.4
Re_arc = 32.4
Re_kpc = 2.576
M_star = 11.26

corr=0.77
gal = 'N4526'
r_e = 2.576 #Re in kpc, Duncan's paper
n_S = 3.6 #Sersic Index, Duncan's paper

data = np.loadtxt(input_file, dtype={'names':('ID', 'RA', 'DEC', 'RV', 'RV_err', 'col', 'PA', 'RAD'), 'formats':('|S20', np.float, np.float, np.float, np.float, np.float, np.float, np.float)})

ID = data['ID']
RA =data['RA']
DEC=data['DEC']
RV=data['RV']
RV_err=data['RV_err']
col=data['col']
RAD=data['RAD']
PA=data['PA']


if friendless==True:
    print "Welcome, rot-sig estimation routine!"
    print "+++++++++++++++++++++++++++++++++"
    print ""
    print "There are ", len(ID), "objects in your dataset, before cleaning"

    no_print = outliers_show(RV, RAD, RA, DEC)
    fRAD = np.empty(len(no_print))
    fRV = np.empty(len(no_print))

    for i in range(len(no_print)):
        print ID[no_print[i]]
        fRAD[i] = RAD[no_print[i]]
        fRV[i] = RV[no_print[i]]

    #clean friendless objects from parent dataset, and update (not writing to file!!!)
    ID = np.delete(ID, list(no_print))
    RA = np.delete(RA, list(no_print))
    DEC = np.delete(DEC, list(no_print))
    RV = np.delete(RV, list(no_print))
    RV_err = np.delete(RV_err, list(no_print))
    col = np.delete(col, list(no_print))
    PA = np.delete(PA, list(no_print))
    RAD = np.delete(RAD, list(no_print))


    print "dataset now has ", len(ID), "objects, after cleaning"
    #return(ID,RA,DEC,RV,RV_err,col,PA,RAD)
ngc = len(RA)

# ORDERING BY GALACTOCENTRIC RADIUS:            #kin.py
# All GCs:          #kin.py
inds=np.argsort(RAD)            #kin.py
ros=RAD[inds]           #kin.py
paos=PA[inds]           #kin.py
rvos=RV[inds]           #kin.py
rveos=RV_err[inds]          #kin.py
            #kin.py
# INITIAL GUESS:            #kin.py
x0=np.zeros(3)          #kin.py
x0[0]=80.           #kin.py
x0[1]=200.          #kin.py
x0[2]=pa_phot           #kin.py

# Bootstrapping:
randind=[]
for i in range(0,ngc):
    randind.append(0)

for j in range (0,nbs-1):
    #print j
    # Randomizing all GCs:
    for i in range(0,ngc):
        randind[i]=random.randint(0,ngc) # assigning random index
    randrs=RAD[randind]
    randpas=PA[randind]
    randrvs=RV[randind]
    randrves=RV_err[randind]

    # ORDERING BY GALACTOCENTRIC RADIUS:
    # ALL GCs:
    inds=np.argsort(randrs)
    ros=randrs[inds]
    paos=randpas[inds]
    rvos=randrvs[inds]
    rveos=randrves[inds]

    # INITIAL GUESS:
    x0=np.zeros(3)
    x0[0]=60.
    x0[1]=130.
    x0[2]=pi

# MINIMIZING USING POWELL'S METHOD for all GCs:
    rad=0.
    rad=sum(ros[0:ngc])/float(ngc)
    powellout=fmin_powell(afunc,x0,args=(paos,v_sys,q_phot),ftol = 1e-6, xtol=1e-6, maxiter = None,full_output=1,disp=0)
    xopt=powellout[0]
    #using last fit as new initial guess:
    x0=xopt
    #print j+1,rad,xopt[0],xopt[1],(xopt[2]/pi*180.),powellout[1]/float(ngc-3)
    out.write(str(j+1)+'\t'+str(rad)+'\t'+str(xopt[0])+'\t'+str(xopt[1])+'\t'+str((xopt[2]/pi*180.))+'\t'+str(powellout[1]/float(ngc-3))+'\n')

#out1=open('FRIENDLESS','w')
#out1.write('#ID \t RA \t DEC \t RV \t RV_err \t col \t PA \t RAD \t \n')
out.close()
out4=open(mass_estW_ISO,'w')
out4.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')
out5=open(mass_estW_RAD,'w')
out5.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')
out6=open(mass_estW_TAN,'w')
out6.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')


#det gamm and alpha parameter
gamm_param = -0.63*M_star+9.81
alp_param = -0.46*M_star+5.29
#define alpha, beta and gamma
gamm = [gamm_param]
alpha = [alp_param]
beta = [[0.], [0.5], [-0.5]]

from tqdm import trange

Vrot = rotation_definition(file=all_rad_file,percentile=50,columns =[2,3])[0]#88.57
err_l = rotation_definition(file=all_rad_file,percentile=16,columns =[2,3])[0]#27.8421455382 #16th percentile
err_u = rotation_definition(file=all_rad_file,percentile=84,columns =[2,3])[0] #84th percentile

RV -= Vsys
RV -= Vrot
RAD = RAD * Re_kpc/Re_arc
#sort data by RAD before analysis
indx = np.argsort(RAD)
RV = RV[indx]
RV_err = RV_err[indx]
RAD = RAD[indx]

N=1000 #number of iterations for Montecarlo simulations
idx = np.nonzero(RAD > 2.*Re_kpc)[0] #returns index of GCs greater than 2R_e
#print idx, RAD.min(),RAD.max(),Re_kpc
mass_est_err0 = np.ones(N)
mass_est_err1 = np.ones(N)
mass_est_err2 = np.ones(N)

for rad_lim in RAD[idx]:
    aRAD = RAD[np.where(np.logical_and(RAD < rad_lim, 1))]
    aRV = RV[np.where(np.logical_and(RAD < rad_lim, 1))]
    aRV_err = RV_err[np.where(np.logical_and(RAD < rad_lim, 1))]
    #real_mass, without Monte Carlo simulations
    real_mass_est0 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[0], rad_lim)
    real_mass_est1 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[1], rad_lim)
    real_mass_est2 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[2], rad_lim)
    #Now Monte Carlo, that makes use of RV_err and sampling without replacement
    for j in range(N):
        scr_RV = np.ones(len(aRV))
        for i in range(len(aRV)):
            scr_RV[i] = np.random.normal(aRV[i], aRV_err[i], 1)
        VELSQ = np.square(scr_RV)
        mass_est_err0[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[0], rad_lim)/np.sqrt(len(aRAD))
        mass_est_err1[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[1], rad_lim)/np.sqrt(len(aRAD))
        mass_est_err2[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[2], rad_lim)/np.sqrt(len(aRAD))
    #print ("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est0, scipy.stats.scoreatpercentile(mass_est_err0[:], 16), scipy.stats.scoreatpercentile(mass_est_err0[:], 84)))
    out4.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est0, scipy.stats.scoreatpercentile(mass_est_err0[:], 16), scipy.stats.scoreatpercentile(mass_est_err0[:], 84)))
    out5.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est1, scipy.stats.scoreatpercentile(mass_est_err1[:], 16), scipy.stats.scoreatpercentile(mass_est_err1[:], 84)))
    out6.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est2, scipy.stats.scoreatpercentile(mass_est_err2[:], 16), scipy.stats.scoreatpercentile(mass_est_err2[:], 84)))
out4.close()
out5.close()
out6.close()
#######################################
# find the masses and max re from the following code!! Also see output.py!!

#######################################
#read input files
#isotropy

ngc, rad, mass_W_I, lmass_err_W_I, umass_err_W_I = np.loadtxt(mass_estW_ISO, unpack=True, usecols=[0,1,2,3,4])
mass_err_W_I = np.maximum(lmass_err_W_I, umass_err_W_I)
#radial orbits
mass_W_R, lmass_err_W_R, umass_err_W_R = np.loadtxt(mass_estW_RAD, unpack=True, usecols=[2,3,4])
mass_err_W_R = np.maximum(lmass_err_W_R, umass_err_W_R)
#tangential orb
mass_W_T, lmass_err_W_T, umass_err_W_T = np.loadtxt(mass_estW_TAN, unpack=True, usecols=[2,3,4])
mass_err_W_T = np.maximum(lmass_err_W_T, umass_err_W_T)


effective_radii_to_investigate=np.append(np.arange(3.,(rad[-1]/r_e),1.),np.round(rad[-1]/r_e,2))
max_rad = effective_radii_to_investigate
# Isotropic 
if mode == 'iso':
    #input_array, corr, re,rad,mode,re_sampled=np.arange(1.,15.,1.)):
    m5 = define_masses_at_re(input_array=mass_W_I,re_sampled=effective_radii_to_investigate, mode='iso')[0]#2.66e+11#mass within 5/8/Rmax 3.52e+11 8.35e+10 
    m5err = define_masses_at_re(input_array=mass_err_W_I,re_sampled=effective_radii_to_investigate,mode='iso')[1]#7.86e+10 #uncertainty

# rad 
elif mode == 'rad':
    m5 = define_masses_at_re(effective_radii_to_investigate, mode='rad')[0]
    m5err = define_masses_at_re(effective_radii_to_investigate,mode='rad')[1]

#tan
elif mode == 'tan':
    m5 = define_masses_at_re(effective_radii_to_investigate, mode='tan')[0]
    m5err = define_masses_at_re(effective_radii_to_investigate,mode='tan')[1]



M_star = 11.26
M_star = 10**M_star
mstar_err = 0.10*M_star
dm_frac = []
dm_frac_err = []
mass_re=[]
mass_re_max=[]
mass_re_err=[]
mass_re_max_err=[]
#obtain parameters and uncertainties on calculated parameters
######################## Needs to loop over all the Vrot values
for i in range(len(max_rad)):
    m_rot5, m_rot5_err, tot_mass5, tot_mass5_err = get_mass(r_e, max_rad[i], Vrot, err_l, err_u, m5[i], m5err[i], corr)
    f_dm5, f_dm5_err = get_f_dm(M_star, mstar_err, max_rad[i], tot_mass5, tot_mass5_err, r_e, n_S)
    dm_frac.append(f_dm5)
    dm_frac_err.append(f_dm5_err)
    mass_re.append(m_rot5)
    mass_re_max.append(tot_mass5)
    mass_re_err.append(m_rot5_err)
    mass_re_max_err.append(tot_mass5_err)
    print("%5s \t %1s \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2f \t %.2f \n" % (str(gal),max_rad[i],m_rot5, m_rot5_err, m5[i], m5err[i], tot_mass5, tot_mass5_err, f_dm5, f_dm5_err))
#"%5s \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2f \t %.2f \n" %
#4526    2.66e+11    7.86e+10    16.75   6.52e+11    1.35e+11
#return (max_rad, m_rot5, m_rot5_err, m5[i], m5err[i], tot_mass5, tot_mass5_err, f_dm5, f_dm5_err)
d ={'re': effective_radii_to_investigate,
    'fDM': dm_frac,
    'fDM_err': dm_frac_err,
    'mtot':tot_mass5,
    'mtot_err':tot_mass5_err,
    'mass_rot':mass_re,
    'mass_rot_err': mass_re_err,
    'mass_press': m5,
    'mass_press_err': m5err}
df = pd.DataFrame(d)

df.to_csv(str(input_file[:-4])+'_'+str(friendless)+'_'+str(mode)+'_no_rot_corr.csv')

Welcome, rot-sig estimation routine!
+++++++++++++++++++++++++++++++++

There are  100 objects in your dataset, before cleaning
dataset now has  100 objects, after cleaning
N4526 	 3.0 	 3.54e+10 	 1.06e+10 	 2.12e+11 	 6.24e+10 	 2.47e+11 	 4.92e+10 	 0.40 	 0.17 

N4526 	 4.0 	 4.72e+10 	 1.41e+10 	 2.39e+11 	 6.09e+10 	 2.86e+11 	 4.91e+10 	 0.45 	 0.13 

N4526 	 5.0 	 5.90e+10 	 1.73e+10 	 2.93e+11 	 7.14e+10 	 3.52e+11 	 5.65e+10 	 0.53 	 0.10 

N4526 	 6.0 	 7.08e+10 	 2.12e+10 	 3.30e+11 	 7.32e+10 	 4.00e+11 	 6.01e+10 	 0.58 	 0.08 

N4526 	 7.0 	 8.26e+10 	 2.52e+10 	 3.77e+11 	 8.08e+10 	 4.59e+11 	 6.70e+10 	 0.62 	 0.07 

N4526 	 8.0 	 9.44e+10 	 2.83e+10 	 4.36e+11 	 9.05e+10 	 5.30e+11 	 7.50e+10 	 0.67 	 0.06 

N4526 	 9.0 	 1.06e+11 	 3.18e+10 	 4.66e+11 	 9.33e+10 	 5.73e+11 	 7.94e+10 	 0.69 	 0.06 

N4526 	 10.0 	 1.18e+11 	 3.56e+10 	 5.28e+11 	 1.04e+11 	 6.46e+11 	 8.73e+10 	 0.73 	 0.05 

N4526 	 11.0 	 1.30e+11 	 3.92e+10 	 5.79e+11 	 1.13e+11 	 7.09e+11 	 9.50

In [ ]:
print rad

In [ ]:
!ls

input_file='NGC4526.cat'
friendless=True
mode ='iso'
#def calculate_dark_matter_fraction(RV_file):
#input_file=RV_file
all_rad_file='ALL_rad_roll_'+input_file
mass_estW_ISO='mass_estW_ISO_'+input_file
mass_estW_RAD='mass_estW_RAD_'+input_file
mass_estW_TAN='mass_estW_TAN_'+input_file


out=open(all_rad_file,'w')
out.write('#it \t\t rad \t\t Vrot \t\t sigma \t\t PAkin \t\t lh/dof \n')

nbs=30
v_sys=617.
pi=math.pi
q_phot=0.24
pa_phot=(113.7)*pi/180.0
RA0 = 188.512856
DEC0 = 7.699524
Vsys = 617.
Q = 0.24
PA0 = 113.7
Dist = 16.4
Re_arc = 32.4
Re_kpc = 2.576
M_star = 11.26

corr=0.77
gal = 'N4526'
r_e = 2.576 #Re in kpc, Duncan's paper
n_S = 3.6 #Sersic Index, Duncan's paper

data = np.loadtxt(input_file, dtype={'names':('ID', 'RA', 'DEC', 'RV', 'RV_err', 'col', 'PA', 'RAD'), 'formats':('|S20', np.float, np.float, np.float, np.float, np.float, np.float, np.float)})

ID = data['ID']
RA =data['RA']
DEC=data['DEC']
RV=data['RV']
RV_err=data['RV_err']
col=data['col']
RAD=data['RAD']
PA=data['PA']


if friendless==True:
    print "Welcome, rot-sig estimation routine!"
    print "+++++++++++++++++++++++++++++++++"
    print ""
    print "There are ", len(ID), "objects in your dataset, before cleaning"

    no_print = outliers_show(RV, RAD, RA, DEC)
    fRAD = np.empty(len(no_print))
    fRV = np.empty(len(no_print))

    for i in range(len(no_print)):
        print ID[no_print[i]]
        fRAD[i] = RAD[no_print[i]]
        fRV[i] = RV[no_print[i]]

    #clean friendless objects from parent dataset, and update (not writing to file!!!)
    ID = np.delete(ID, list(no_print))
    RA = np.delete(RA, list(no_print))
    DEC = np.delete(DEC, list(no_print))
    RV = np.delete(RV, list(no_print))
    RV_err = np.delete(RV_err, list(no_print))
    col = np.delete(col, list(no_print))
    PA = np.delete(PA, list(no_print))
    RAD = np.delete(RAD, list(no_print))


    print "dataset now has ", len(ID), "objects, after cleaning"
    #return(ID,RA,DEC,RV,RV_err,col,PA,RAD)
ngc = len(RA)

# ORDERING BY GALACTOCENTRIC RADIUS:            #kin.py
# All GCs:          #kin.py
inds=np.argsort(RAD)            #kin.py
ros=RAD[inds]           #kin.py
paos=PA[inds]           #kin.py
rvos=RV[inds]           #kin.py
rveos=RV_err[inds]          #kin.py
            #kin.py
# INITIAL GUESS:            #kin.py
x0=np.zeros(3)          #kin.py
x0[0]=80.           #kin.py
x0[1]=200.          #kin.py
x0[2]=pa_phot           #kin.py

# Bootstrapping:
randind=[]
for i in range(0,ngc):
    randind.append(0)

for j in range (0,nbs-1):
    #print j
    # Randomizing all GCs:
    for i in range(0,ngc):
        randind[i]=random.randint(0,ngc) # assigning random index
    randrs=RAD[randind]
    randpas=PA[randind]
    randrvs=RV[randind]
    randrves=RV_err[randind]

    # ORDERING BY GALACTOCENTRIC RADIUS:
    # ALL GCs:
    inds=np.argsort(randrs)
    ros=randrs[inds]
    paos=randpas[inds]
    rvos=randrvs[inds]
    rveos=randrves[inds]

    # INITIAL GUESS:
    x0=np.zeros(3)
    x0[0]=60.
    x0[1]=130.
    x0[2]=pi

# MINIMIZING USING POWELL'S METHOD for all GCs:
    rad=0.
    rad=sum(ros[0:ngc])/float(ngc)
    powellout=fmin_powell(afunc,x0,ftol = 1e-6, xtol=1e-6, maxiter = None,full_output=1,disp=0)
    xopt=powellout[0]
    #using last fit as new initial guess:
    x0=xopt
    #print j+1,rad,xopt[0],xopt[1],(xopt[2]/pi*180.),powellout[1]/float(ngc-3)
    out.write(str(j+1)+'\t'+str(rad)+'\t'+str(xopt[0])+'\t'+str(xopt[1])+'\t'+str((xopt[2]/pi*180.))+'\t'+str(powellout[1]/float(ngc-3))+'\n')

#out1=open('FRIENDLESS','w')
#out1.write('#ID \t RA \t DEC \t RV \t RV_err \t col \t PA \t RAD \t \n')
out.close()
out4=open(mass_estW_ISO,'w')
out4.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')
out5=open(mass_estW_RAD,'w')
out5.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')
out6=open(mass_estW_TAN,'w')
out6.write('#N_GC \t max_rad \t mass \t l_mass \t u_mass \t \n')

#det gamm and alpha parameter
gamm_param = -0.63*M_star+9.81
alp_param = -0.46*M_star+5.29
#define alpha, beta and gamma
gamm = [gamm_param]
alpha = [alp_param]
beta = [[0.], [0.5], [-0.5]]


Vrot = rotation_definition(file=all_rad_file,percentile=50,columns =[2,3])[0]#88.57
err_l = rotation_definition(file=all_rad_file,percentile=16,columns =[2,3])[0]#27.8421455382 #16th percentile
err_u = rotation_definition(file=all_rad_file,percentile=84,columns =[2,3])[0] #84th percentile

RV -= Vsys
RV -= Vrot
RAD = RAD * Re_kpc/Re_arc
#sort data by RAD before analysis
indx = np.argsort(RAD)
RV = RV[indx]
RV_err = RV_err[indx]
RAD = RAD[indx]

N=1000 #number of iterations for Montecarlo simulations
idx = np.nonzero(RAD > 2.*Re_kpc)[0] #returns index of GCs greater than 2R_e

mass_est_err0 = np.ones(N)
mass_est_err1 = np.ones(N)
mass_est_err2 = np.ones(N)

for k in range(len(idx)):
    rad_lim = RAD[idx[k]]
    aRAD = RAD[np.where(np.logical_and(RAD < rad_lim, 1))]
    aRV = RV[np.where(np.logical_and(RAD < rad_lim, 1))]
    aRV_err = RV_err[np.where(np.logical_and(RAD < rad_lim, 1))]
    #real_mass, without Monte Carlo simulations
    real_mass_est0 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[0], rad_lim)
    real_mass_est1 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[1], rad_lim)
    real_mass_est2 = Watkins(aRAD, np.square(aRV), alpha, gamm, beta[2], rad_lim)
    #Now Monte Carlo, that makes use of RV_err and sampling without replacement
    for j in range(N):
        scr_RV = np.ones(len(aRV))
        for i in range(len(aRV)):
            scr_RV[i] = np.random.normal(aRV[i], aRV_err[i], 1)
        VELSQ = np.square(scr_RV)
        mass_est_err0[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[0], rad_lim)/np.sqrt(len(aRAD))
        mass_est_err1[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[1], rad_lim)/np.sqrt(len(aRAD))
        mass_est_err2[j] = 1.402*Watkins(aRAD, VELSQ, alpha, gamm, beta[2], rad_lim)/np.sqrt(len(aRAD))

    out4.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est0, scipy.stats.scoreatpercentile(mass_est_err0[:], 16), scipy.stats.scoreatpercentile(mass_est_err0[:], 84)))
    out5.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est1, scipy.stats.scoreatpercentile(mass_est_err1[:], 16), scipy.stats.scoreatpercentile(mass_est_err1[:], 84)))
    out6.write("%3i \t %.3f \t %.5e \t %.5e \t %.5e \n" % (len(aRAD), rad_lim, real_mass_est2, scipy.stats.scoreatpercentile(mass_est_err2[:], 16), scipy.stats.scoreatpercentile(mass_est_err2[:], 84)))
out4.close()
out5.close()
out6.close()
#######################################
# find the masses and max re from the following code!! Also see output.py!!

#######################################
#read input files
#isotropy

ngc, rad, mass_W_I, lmass_err_W_I, umass_err_W_I = np.loadtxt(mass_estW_ISO, unpack=True, usecols=[0,1,2,3,4])
mass_err_W_I = np.maximum(lmass_err_W_I, umass_err_W_I)
#radial orbits
mass_W_R, lmass_err_W_R, umass_err_W_R = np.loadtxt(mass_estW_RAD, unpack=True, usecols=[2,3,4])
mass_err_W_R = np.maximum(lmass_err_W_R, umass_err_W_R)
#tangential orb
mass_W_T, lmass_err_W_T, umass_err_W_T = np.loadtxt(mass_estW_TAN, unpack=True, usecols=[2,3,4])
mass_err_W_T = np.maximum(lmass_err_W_T, umass_err_W_T)


effective_radii_to_investigate=np.append(np.arange(3.,(rad[-1]/r_e),1.),np.round(rad[-1]/r_e,2))
max_rad = effective_radii_to_investigate
# Isotropic 
if mode == 'iso':
    m5 = define_masses_at_re(effective_radii_to_investigate, mode='iso')[0]#2.66e+11#mass within 5/8/Rmax 3.52e+11 8.35e+10 
    m5err = define_masses_at_re(effective_radii_to_investigate,mode='iso')[1]#7.86e+10 #uncertainty

# rad 
elif mode == 'rad':
    m5 = define_masses_at_re(effective_radii_to_investigate, mode='rad')[0]
    m5err = define_masses_at_re(effective_radii_to_investigate,mode='rad')[1]

#tan
elif mode == 'tan':
    m5 = define_masses_at_re(effective_radii_to_investigate, mode='tan')[0]
    m5err = define_masses_at_re(effective_radii_to_investigate,mode='tan')[1]



M_star = 11.26
M_star = 10**M_star
mstar_err = 0.10*M_star
dm_frac = []
dm_frac_err = []
mass_re=[]
mass_re_max=[]
mass_re_err=[]
mass_re_max_err=[]
#obtain parameters and uncertainties on calculated parameters
######################## Needs to loop over all the Vrot values
for i in range(len(max_rad)):
    m_rot5, m_rot5_err, tot_mass5, tot_mass5_err = get_mass(r_e, max_rad[i], Vrot, err_l, err_u, m5[i], m5err[i], corr)
    f_dm5, f_dm5_err = get_f_dm(M_star, mstar_err, max_rad[i], tot_mass5, tot_mass5_err, r_e, n_S)
    dm_frac.append(f_dm5)
    dm_frac_err.append(f_dm5_err)
    mass_re.append(m_rot5)
    mass_re_max.append(tot_mass5)
    mass_re_err.append(m_rot5_err)
    mass_re_max_err.append(tot_mass5_err)
    print("%5s \t %1s \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2f \t %.2f \n" % (str(gal),max_rad[i],m_rot5, m_rot5_err, m5[i], m5err[i], tot_mass5, tot_mass5_err, f_dm5, f_dm5_err))
#"%5s \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2e \t %.2f \t %.2f \n" %
#4526    2.66e+11    7.86e+10    16.75   6.52e+11    1.35e+11
#return (max_rad, m_rot5, m_rot5_err, m5[i], m5err[i], tot_mass5, tot_mass5_err, f_dm5, f_dm5_err)
d ={'re': effective_radii_to_investigate,
    'fDM': dm_frac,
    'fDM_err': dm_frac_err,
    'mtot':tot_mass5,
    'mtot_err':tot_mass5_err,
    'mass_rot':mass_re,
    'mass_rot_err': mass_re_err,
    'mass_press': m5,
    'mass_press_err': m5err}
df = pd.DataFrame(d)

